In [1]:
%pylab inline
import skimage.io as sk
import imageio
import ipywidgets as wdg
import skan
from pathlib import Path
import pandas as pd

In /home/rishabhstein/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/rishabhstein/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/rishabhstein/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/rishabhstein/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/rishabhstein/anaconda3/lib/

Populating the interactive namespace from numpy and matplotlib


In [2]:
path = Path('/media/rishabhstein/SSD/Data_for_testing_the definition_of_Threshold/')
fsuffix = 'WB10_diff72_downscaledby2.tif' #It can be a list in Path folder


file = Path(path.resolve(), fsuffix)
img = sk.imread(str(file), as_gray=False)

### Choosing a point of wormhole to extract it from grayscale difference image

In [3]:
%matplotlib tk
fig = figure(figsize=(7,7))
z = 590
imshow(img[z], cmap="gray")
x = 0; y = 0
def onclick(event):
    global x
    global y
    close()
    x = int(floor(event.xdata))
    y = int(floor(event.ydata))
    print(x,y)

ka = fig.canvas.mpl_connect('button_press_event', onclick)

253 333


In [4]:
point = [z, x, y]

In [187]:
import sys
sys.path.append('../')
import lib.Geometric_properties_of_wormhole as GPM

In [188]:
#Reload modules
import importlib
importlib.reload(GPM)

<module 'lib.Geometric_properties_of_wormhole' from '../lib/Geometric_properties_of_wormhole.py'>

## Without Median Filter

### Extraction of Wormhole

In [205]:
list_threshold = [545.0, 665.0, 825.0, 1025.0]
properties = [];

for th_value in list_threshold:
    
    sample = GPM.Wormhole_from_tomography(img, suffix = fsuffix, voxel_size = 35, downscaling_factor=2)
    # Finding the wormhole
    Vol_unfill, wormhole_unfilled = sample.Connected_components_extraction(th_value=th_value, point=point)

    #Filling the holes and calculatin volume
    Vol_fill, wormhole_filled = sample.Fill_holes_in_wormhole()

    #Calculating Surface area and Volume in voxels as well as cm scale
    SA = sample.Surface_area_Legland_et_al()
    Vol_cm, SA_cm = sample.Volume_and_Surface_area_Cm2()


    #Skeletonization and its properties
    _ = sample.Skeletonise_the_wormhole()

    properties.append(sample.info)


Finding dominant wormhole
%=====================Looking for Inlet node=========================%

Node-0is not connected to main wormhole
Node-2 is connected and chosen as source node
%=====================Looking for Outlet node=========================%


 Node-14752 is connected and chosen as outlet node
Finding dominant wormhole
%=====================Looking for Inlet node=========================%

Node-0is not connected to main wormhole
Node-1 is connected and chosen as source node
%=====================Looking for Outlet node=========================%


 Node-9497 is connected and chosen as outlet node
Finding dominant wormhole
%=====================Looking for Inlet node=========================%

Node-0is not connected to main wormhole
Node-1 is connected and chosen as source node
%=====================Looking for Outlet node=========================%


 Node-6576 is connected and chosen as outlet node
Finding dominant wormhole
%=====================Looking for Inlet node=====

In [206]:
properties

[{'File_name': 'WB10_diff72_downscaledby2.tif',
  'voxel_size': 35,
  'downscaling_factor': 2,
  'Volume_in_voxel': 279384,
  'Volume in cm3': 0.095828712,
  'Surface_area_in_voxel': 133360.758320059,
  'Surface_area_in_cm2': 0.0653467715768289,
  'Length_of_wormhole': 1492.0670164589856,
  'Tortuosity': 1.4787581927244653,
  'Wastefullness': 23.53528255168303},
 {'File_name': 'WB10_diff72_downscaledby2.tif',
  'voxel_size': 35,
  'downscaling_factor': 2,
  'Volume_in_voxel': 250730,
  'Volume in cm3': 0.08600039,
  'Surface_area_in_voxel': 110362.09290631153,
  'Surface_area_in_cm2': 0.05407742552409266,
  'Length_of_wormhole': 1489.0981265227713,
  'Tortuosity': 1.4758157844626079,
  'Wastefullness': 14.671704795064976},
 {'File_name': 'WB10_diff72_downscaledby2.tif',
  'voxel_size': 35,
  'downscaling_factor': 2,
  'Volume_in_voxel': 223384,
  'Volume in cm3': 0.076620712,
  'Surface_area_in_voxel': 97504.3696867419,
  'Surface_area_in_cm2': 0.047777141146503534,
  'Length_of_wormho

In [200]:
sample.info

{'File_name': 'WB10_diff72_downscaledby2.tif',
 'voxel_size': 35,
 'downscaling_factor': 2,
 'Volume_in_voxel': 276074,
 'Volume in cm3': 0.094693382,
 'Surface_area_in_voxel': 130600.38768812447,
 'Surface_area_in_cm2': 0.063994189967181,
 'Length_of_wormhole': 1491.5994547586529,
 'Tortuosity': 1.4782948015447501,
 'Wastefullness': 21.970640969207444}

### Save tiff file

In [1]:
from tifffile import imsave
imsave(fsuffix.strip('.tif') + '_skeletonisez.tif', skelton)

## With Median filter

In [190]:
from scipy import ndimage
filterd_image = ndimage.median_filter(sample.filled_wormhole, size = 2)

In [99]:
from tifffile import imsave
imsave(fsuffix.strip('.tif') + '_median3D.tif', filterd_image)

In [191]:
fsuffix_new = fsuffix.strip('.tif') + '_median3D.tif'
filtered_sample = GPM.Wormhole_from_tomography(filterd_image, suffix = fsuffix_new, voxel_size = 35, downscaling_factor=2)
Vol_fill, wormhole_filled = filtered_sample.Fill_holes_in_wormhole_external_image(filterd_image, if_update_class=True)
SA = filtered_sample.Surface_area_Legland_et_al()
Vol_cm, SA_cm = filtered_sample.Volume_and_Surface_area_Cm2()

In [192]:
_ = filtered_sample.Skeletonise_the_wormhole()

Finding dominant wormhole
%=====================Looking for Inlet node=========================%

Node-0is not connected to main wormhole
Node-1 is connected and chosen as source node
%=====================Looking for Outlet node=========================%


 Node-5815 is connected and chosen as outlet node


In [193]:
filtered_sample.info

{'File_name': 'WB10_diff72_downscaledby2_median3D.tif',
 'voxel_size': 35,
 'downscaling_factor': 2,
 'Volume_in_voxel': 285704,
 'Volume in cm3': 0.097996472,
 'Surface_area_in_voxel': 105565.50688884944,
 'Surface_area_in_cm2': 0.051727098375536226,
 'Length_of_wormhole': 1528.0920045700566,
 'Tortuosity': 1.5144618479386092,
 'Wastefullness': 8.139148789058495}

In [197]:
1496*2*35*1e-4


10.472000000000001